We repeat the test, with Aspen-9 backend.

In [ ]:
import numpy as np

from utils import bf
from QVC import PyquilVariationalClassifier
from pyquil_circuits import PauliFeatureMap, VariationalCircuit

In [2]:
# Define quantum feature map
qfm = PauliFeatureMap(num_qubits=3, rep=2)
vc = VariationalCircuit(num_qubits=3, rep=2)
qvc = PyquilVariationalClassifier(qfm, vc, bool_ftn=bf, use_bias=False)

In [5]:
# Load Data
train_data = np.load('./npy_files/TrainData_zzpfmc12_pyquil.npy')
test_data = np.load('./npy_files/TestData_zzpfmc12_pyquil.npy')
train_labels = np.load('./npy_files/TrainLabels_zzpfmc12_pyquil.npy')
test_labels = np.load('./npy_files/TestLabels_zzpfmc12_pyquil.npy')
opt_param = np.load('./npy_files/Optimal_param_zzpfmc12_qiskit.npy')

qvc.optimal_params = opt_param

[ 1.56636433 -0.72325934 -0.94454689  1.72973839  0.91390905  1.47260758
  1.40753721  1.61813452  1.65474997  1.65748732  2.70052991  2.21620275
  0.77342995 -0.57277874 -1.83137667  0.5672403 ]


In [6]:
test_acc, _, test_correct = qvc.test(test_data, (-1)**test_labels, verbose=True, backend='Aspen-9')
print(f"Test accuracy     = {test_acc}")

In [8]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
colors = ['tab:blue', 'tab:red']
correct_dp = list()
false_dp = list()
for dp, l in zip(test_data[:len(test_correct)], test_correct):
    if l :
        correct_dp.append(dp)
    else:
        false_dp.append(dp)
correct_dp = np.array([np.array(d) for d in correct_dp]).swapaxes(0,1)
false_dp = np.array([np.array(d) for d in false_dp]).swapaxes(0,1)
ax.scatter(correct_dp[0], correct_dp[1], correct_dp[2],
           c = colors[0],
           label='correct')
ax.scatter(false_dp[0], false_dp[1], false_dp[2],
           c = colors[1],
           label='false')
ax.set_xlabel('pc1')
ax.set_ylabel('pc2')
ax.set_zlabel('pc3')
fig.legend(bbox_to_anchor=(1,1))
plt.savefig('./figures/zzzpfm_c12_class_test_qpu.png')

0.3333333333333333
